In [1]:
import os
import ftplib
import datetime
import requests
import rasterio
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from rasterio.mask import mask
from scipy.spatial import cKDTree
from shapely.geometry import Point
# from datetime import datetime, timedelta
from mpl_toolkits.basemap import Basemap
from rasterio.transform import from_origin
from netCDF4 import Dataset, date2num, num2date

os.environ["PROJ_LIB"] = "C:/Users/2ndba/anaconda3/Library/share/proj"

DATA PREPARATION
Obtain the data and save it in this folder.

In [2]:
path_nc_file = "../data/row"
path_nc_row = "../repository/pre-processing/row"
path_modified = "../repository/pre-processing/result-row"

pch_tabular = "../data/tabular/data_pch_balai.xlsx"

RETRIEVE NETCDF DATA FROM THE FTP SERVER<br>

<p>
Obtain data for day -1 to use for tomorrow's predictions.
</p>

In [3]:
# konfigurasi ftp
ftp_host = os.getenv("HOST")
ftp_user = os.getenv("USER")
ftp_password = os.getenv("PASSWORD")
cycle = "12"

# FTP functions
def connect_ftp():
    ftp = ftplib.FTP(ftp_host)
    ftp.login(ftp_user, ftp_password)
    ftp.cwd("/")
    return ftp

def download_file_from_ftp(ftp, filename):
    try:
        file_list = ftp.nlst()
        if filename in file_list:
            local_file_path = os.path.join(path_nc_file, filename)
            if not os.path.exists(local_file_path):
                with open(local_file_path, "wb") as local_file:
                    ftp.retrbinary(f"RETR {filename}", local_file.write)
                print(f"Download successfully {filename}")
            else:
                print(f"File {filename} is available")
            return local_file_path
    except Exception:
        print("File is corrupted, and there is nothing that can be done.")
        return None

def download_latest_file_from_ftp(ftp):
    file_list = ftp.nlst()
    if file_list:
        latest_file = sorted(file_list)[-1]
        return download_file_from_ftp(ftp, latest_file)
    return None

# Download file .nc
today = datetime.date.today() - datetime.timedelta(days=2)
filename = f"ECMWF.0125.{today.strftime('%Y%m%d')}{cycle}00.PREC.nc"
print("Downloading:", filename)

ftp = connect_ftp()
if ftp:
    local_file_path = download_file_from_ftp(ftp, filename) or download_latest_file_from_ftp(ftp)
    ftp.quit()
else:
    print("Cannot connect to FTP server")

if local_file_path is None:
    print("File is currently unavailable for download.")
    exit()

Downloading: ECMWF.0125.202501261200.PREC.nc
Download successfully ECMWF.0125.202501261200.PREC.nc


READ THE NETCDF DATA<br/>

<p>
Data in netCDF format that has been downloaded contains four main parameters, namely lat (latitude), which represent latitude coordinates, lon (longitude) as longitude coordinates, tp (total precipitation) which shows total rainfall, and time which records the time the data was taken
</p>

<p>
Gunakan data H+1 dan H+2 untuk prediksi curah hujan
</p>

In [4]:
# Baca data .nc
data = Dataset(local_file_path)

lat = data.variables['lat'][:]
lon = data.variables['lon'][:]
prec = data.variables['tp'][:,0,:,:]
time = data.variables['time'][:]

dates = num2date(time, data.variables['time'].units)
print((dates[7]+datetime.timedelta(hours=7)).strftime('%Y-%m-%d %H:%M:%S WIB'))
print((dates[15]+datetime.timedelta(hours=7)).strftime('%Y-%m-%d %H:%M:%S WIB'))

2025-01-27 16:00:00 WIB
2025-01-28 16:00:00 WIB


Proses pengolahan data curah hujan dimulai dengan menginisialisasi data hujan sebelum mengubahnya dari format akumulasi menjadi format interval waktu. Selanjutnya, data curah hujan akumulasi diubah menjadi data interval dengan menghitung selisih antara waktu saat ini dan waktu sebelumnya. Setelah itu, data yang berbentuk 3D (waktu, lintang, bujur) diubah menjadi format 4D dengan menambahkan dimensi tambahan agar sesuai dengan format permanen NetCDF. Akhirnya, data hasil olahan tersebut ditulis ulang ke dalam file NetCDF untuk keperluan penyimpanan dan analisis lebih lanjut.

In [5]:
xrain = prec
print(np.array_equal(xrain, prec))

for time in range (len(dates)):
    for latitude in range(len(lat)) :
        for longitude in range (len(lon)) :
            if (time<=0):
                if (xrain[time, latitude, longitude]<=0):
                    xrain[time, latitude, longitude] == 0
            elif(time>0):
                if (xrain[time, latitude, longitude]<0):
                    xrain[time, latitude, longitude] = xrain[time-1, latitude, longitude]
                if (xrain[time, latitude, longitude]-xrain[time-1, latitude, longitude]<0) :
                    xrain[time, latitude, longitude] = xrain[time-1, latitude, longitude]

rain_interval = np.empty((len(dates),len(lat),len(lon)))

rain_interval[0,:,:] = xrain[0,:,:]
for i in range (1,len(dates)):
    rain_interval[i,:,:] = xrain[i,:,:]-xrain[i-1,:,:]

rain_four_demantion = rain_interval.reshape(len(dates), 1, len(lat), len(lon))

rain_interval_result = xr.open_dataset(local_file_path)
rain_interval_result['tp'].values = rain_four_demantion
rain_interval_result = rain_interval_result.assign_coords(time=("time",rain_interval_result['time'].values + np.timedelta64(7,'h')))

output_rewrite = f"ECMWF_new.0125.{today.strftime('%Y%m%d')}{cycle}00.PREC.nc"
output_path = os.path.join(path_nc_row, output_rewrite)
rain_interval_result.to_netcdf(output_path)
print (rain_interval_result)

True
<xarray.Dataset> Size: 43MB
Dimensions:  (lat: 185, lon: 449, lev: 1, time: 65)
Coordinates:
  * lat      (lat) float64 1kB 9.0 8.875 8.75 8.625 ... -13.75 -13.88 -14.0
  * lon      (lon) float64 4kB 92.0 92.12 92.25 92.38 ... 147.8 147.9 148.0
  * lev      (lev) float64 8B 1.013e+03
  * time     (time) datetime64[ns] 520B 2025-01-26T19:00:00 ... 2025-02-05T19...
Data variables:
    tp       (time, lev, lat, lon) float64 43MB 0.0 0.0 0.0 ... 0.8438 0.75
Attributes:
    title:        IFS Precipitation
    conventions:  COARDS
    datatype:     Grid
    cachesize:    626240 bytes


In [6]:
result_file_name = f"ECMWF_new_3d.0125.{today.strftime('%Y%m%d')}{cycle}00.PREC.nc"

# gabungkan path dengan nama file
file_path = os.path.join(path_modified, result_file_name)

# buat file NetCDF baru
new_data_interval = Dataset(file_path, 'w', format='NETCDF4')
print(f"File {result_file_name} berhasil dibuat")
print(data.variables)

File ECMWF_new_3d.0125.202501261200.PREC.nc berhasil dibuat
{'lat': <class 'netCDF4.Variable'>
float64 lat(lat)
    grads_dim: y
    grads_mapping: linear
    grads_size: 185
    units: degrees_north
    long_name: latitude
    minimum: -14.0
    maximum: 9.0
    resolution: -0.125
unlimited dimensions: 
current shape = (185,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lon': <class 'netCDF4.Variable'>
float64 lon(lon)
    grads_dim: x
    grads_mapping: linear
    grads_size: 449
    units: degrees_east
    long_name: longitude
    minimum: 92.0
    maximum: 148.0
    resolution: 0.125
unlimited dimensions: 
current shape = (449,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lev': <class 'netCDF4.Variable'>
float64 lev(lev)
    grads_dim: z
    grads_mapping: levels
    units: millibar
    long_name: altitude
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'time': <class 'netCDF4.Variable'>


In [7]:
# mengambil nilai untuk parameter rain, lat, long, time 
# rain diambil dari hasil interval rain_four_demantion
# lat, lon dan time diambil dari data asli netcdf
interval_for_rain = rain_interval[:48,:,:]
interval_for_latitude = data.variables['lat'][:]
interval_for_longitude = data.variables['lon'][:]
interval_for_time = data.variables['time'][:48]

# membuat demensi untuk netCDF
new_data_interval.createDimension('lon', len(interval_for_longitude))
new_data_interval.createDimension('lat', len(interval_for_latitude))
new_data_interval.createDimension('time', len(interval_for_time))

# membuat variabel untuk netCDF
new_interval_for_longitude = new_data_interval.createVariable('lon', 'f4', 'lon')
new_interval_for_latitude = new_data_interval.createVariable('lat', 'f4', 'lat')
new_interval_for_rain = new_data_interval.createVariable('rain', 'f4', ('time', 'lat', 'lon'))
new_interval_for_time = new_data_interval.createVariable('time', 'i4', 'time')

# definisikan variabel dan simpan menjadi netCDF baru
new_interval_for_longitude[:] = interval_for_longitude[:]
new_interval_for_latitude[:] = interval_for_latitude[:]
new_interval_for_rain[:, :, :] = rain_interval[:48,:,:]
new_interval_for_time[:] = interval_for_time + 7

# tambhin informasi umum
new_data_interval.description = "ECMWF from BMKG modified by Gerardus David"
new_data_interval.history = "Cretaed" + today.strftime("%d%m%y")

new_interval_for_longitude.units = "degree_east"
new_interval_for_latitude.units = "degree_north"
new_interval_for_time.units = "hours since "+(dates[0]+datetime.timedelta(hours=7)).strftime('%Y-%m-%d ')+str(cycle)+":00:00"
new_interval_for_rain.units = "mm"

new_data_interval.close()

In [8]:
new_interval_data_path = f"../repository/pre-processing/result-row\ECMWF_new_3d.0125.{today.strftime('%Y%m%d')}{cycle}00.PREC.nc"

n = 0 

interval_data = Dataset(new_interval_data_path)
interval_data.variables

{'lon': <class 'netCDF4.Variable'>
 float32 lon(lon)
     units: degree_east
 unlimited dimensions: 
 current shape = (449,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lat': <class 'netCDF4.Variable'>
 float32 lat(lat)
     units: degree_north
 unlimited dimensions: 
 current shape = (185,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'rain': <class 'netCDF4.Variable'>
 float32 rain(time, lat, lon)
     units: mm
 unlimited dimensions: 
 current shape = (48, 185, 449)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'time': <class 'netCDF4.Variable'>
 int32 time(time)
     units: hours since 2025-01-26 12:00:00
 unlimited dimensions: 
 current shape = (48,)
 filling on, default _FillValue of -2147483647 used}

In [9]:
interval_latitude = interval_data.variables["lat"][:]
interval_longitude = interval_data.variables["lon"][:]
interval_prec = interval_data.variables["rain"][:,:,:]

interval_time = interval_data.variables["time"][:]
interval_dates = num2date(interval_time, interval_data.variables["time"].units)

print((interval_dates[10+n]).strftime('%Y-%m-%d %H:%M:%S WIB'))
print((interval_dates[16+n]).strftime('%Y-%m-%d %H:%M:%S WIB'))

2025-01-28 01:00:00 WIB
2025-01-28 19:00:00 WIB


BACA DATA TABULAR UNTUK MEMBUAT DATA TABEL

In [ ]:
pd.set_option("display.max.columns", None)
grid = pd.read_excel(pch_tabular)
grid

In [11]:
#Get Data index lon, lat, balai, ws, kota
grid_long = grid['idx_long'].to_numpy()
grid_lat = grid['idx_lat'].to_numpy()
longitude_r = grid['long_data']
latitude_r = grid['lat_data']
latitude_prod = grid['lat_prod']
longitude_prod = grid['long_prod']
pulau = grid['pulau']
balai= grid['balai']
kode_balai = grid['kode_balai']
ws = grid ['wilayah_sungai']
das = grid['das']
prov=grid["provinsi"]
kota = grid['kabkot']
wilayah = grid['wilayah']
latshape = grid_lat.shape[0]
latshape

46995

In [12]:
# Forecasting 1 day ahead
for k in range (11+n,19+n):
    print((dates[k]).strftime("%Y%m%d%H"))
    idx_t=(dates[k]).strftime("%Y%m%d%H")
    if (k==19+n):
        globals()['hujanharian_'+(idx_t)] = interval_prec[11+n,:,:]
    else:
        globals()['hujanharian_'+(idx_t)] = interval_prec[11+n:k+1,:,:].sum(axis=0)

2025012721
2025012800
2025012803
2025012806
2025012809
2025012812
2025012815
2025012818


In [13]:
# Forecasting 2 days ahead
for k in range (11+8+n,19+8+n):
    print((dates[k]).strftime("%Y%m%d%H"))
    idx_t = (dates[k]).strftime("%Y%m%d%H")
    if (k == 19 + 8 + n):
        globals()['hujanharian_'+(idx_t)] = prec[11+8+n,:,:]
    else:
        globals()['hujanharian_'+(idx_t)] = prec[11+8+n:k+1,:,:].sum(axis=0)

2025012821
2025012900
2025012903
2025012906
2025012909
2025012912
2025012915
2025012918


In [ ]:
kolom = ['long_prod', 'lat_prod', 'tanggal', 'longitude','latitude','pulau', 'kode_balai', 'balai','das','provinsi','kabkot','wilayah']

df_waspada = pd.DataFrame(columns=kolom)

for tab in range (latshape) :
    gridlat = grid_lat[tab]
    gridlon = grid_long[tab]
    for k in range (11 + n, 27 + n):
        #utk cek awal mulai waspada
        idx_t = (dates[k]).strftime("%Y%m%d%H")
        idx_h = (dates[k]).strftime("%H:00")
        hujan_cek = globals()['hujanharian_'+(idx_t)]
        
        #untuk tanggal status siaga banjir dan pengecekan status akhir siaga banjir di tiap grid
        i_idx = 11 + n if k < 19 + n else 19 + n
        tanggal = (dates[i_idx]).strftime("%d %B %Y")
        
        if (hujan_cek[gridlat, gridlon] >= 0.5):
            df = pd.DataFrame([{'tanggal':tanggal, 'long_prod':longitude_prod[tab], 'lat_prod':latitude_prod[tab], 'longitude':longitude_r[tab],'latitude':latitude_r[tab], 'pulau':pulau[tab], 'kode_balai':kode_balai[tab], 'balai':balai[tab],\
                             'das':das[tab],'provinsi':prov[tab],'kabkot':kota[tab],'wilayah':wilayah[tab]\
                              ,'waktu_mulai':idx_h}])
            i_idx = (11+n) if k < (19+n) else (19+n)
            for i in range (i_idx, i_idx + 8):
                idx_t = (dates[i]).strftime("%Y%m%d%H")
                idx_h = (dates[i]).strftime("%H:00")
                df['ch_' + idx_h] = globals()['hujanharian_'+(idx_t)][gridlat, gridlon]
            
            kelas=globals()['hujanharian_'+(idx_t)][gridlat, gridlon]
            if (0.5 < kelas <= 20):
                status="1" #HUJAN RINGAN
            elif(20 < kelas <= 50):
                status="2" #HUJAN SEDANG
            elif(50 < kelas <=100):
                status="3" #HUJAN LEBAT
            elif(100 < kelas <= 150):
                status="4" #HUJAN SANGAT LEBAT
            elif(kelas > 150):
                status="5" #HUJAN EKSTREM
                
            df["klasifikasi_hujan"] = status
            
            status_cek = globals()['hujanharian_'+(idx_t)][gridlat, gridlon]
            if (0.5 < status_cek <= 50):
                status_1="1" #AMAN
            elif(50 < status_cek <= 75):
                status_1="2" #WASPADA
            elif(75 < status_cek <= 100):
                status_1="3" #SIAGA
            elif(status_cek > 100):
                status_1="4" #AWAS
            
            df["status_akhir"] = status_1
            
            df_waspada = pd.concat([df_waspada, df])
            break
        else:
            continue

df = df_waspada.sort_values(by="tanggal")
df = df.set_index("tanggal")
print (df)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('../repository/pre-processing/accumulation/accum_'+\
                        (dates[11+n]).strftime('%m%d%Y')+'_' #%Y%m%d
                        +(dates[26+n]).strftime('%m%d%Y')+'.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name='Akumulasi Berjalan')
writer.close()

print('Done..')

In [ ]:
# data = pd.read_excel("../new-repository/pre-processing/accumulation/accum_01172025_01192025.xlsx")
data = pd.read_excel(
    f"../repository/pre-processing/accumulation/accum_{(dates[11+n]).strftime('%m%d%Y')}_{(dates[26+n]).strftime('%m%d%Y')}.xlsx"
)

# data['tanggal'] = pd.to_timedelta(data['tanggal'])

print(data)

In [ ]:
from datetime import datetime, timedelta

tanggal_hari_ini = datetime.now()
tanggal_besok = tanggal_hari_ini + timedelta(days=1)

# Formatkan tanggal besok ke string
tanggal_besok_str = tanggal_besok.strftime("%d %B %Y")
print("Tanggal besok:", tanggal_besok_str)

data_tanggal_besok = data[data['tanggal'] == tanggal_besok_str]
data_tanggal_besok


In [ ]:
data_tanggal_besok_sorted = data_tanggal_besok.sort_values(by='ch_01:00', ascending=False)
data_tanggal_besok_sorted